In [35]:
# !pip install -q flwr
# !pip install -U ipywidgets
# !pip install -U flwr["simulation"]

In [36]:
import medmnist
from medmnist import INFO, Evaluator
from torchvision import transforms
from torchvision.transforms import ToTensor, Lambda
import torch.utils.data as data
from torchvision import models
import torch
import tqdm
from collections import OrderedDict
# from typing import List, Tuple
from typing import Dict, List, Optional, Tuple
import numpy as np 
from torch.utils.data import DataLoader, random_split
import flwr as fl
from torch import nn
import torch.nn.functional as F
from flwr.common import Metrics
import random


DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


## Dataset Loading

In [37]:
data_flag = 'chestmnist'


def load_datasets(num_clients: int, batch_size: int):
    
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[.5], std=[.5])
    ])
 
    data_flag = 'chestmnist'
    info = INFO[data_flag]
    DataClass = getattr(medmnist, info['python_class'])


    trainset = DataClass(split='train', transform=transform, target_transform=lambda y:torch.tensor([0,1]) if 1 in y else torch.tensor([1,0]), download=True)
    testset = DataClass(split='test', transform=transform, target_transform=lambda y:torch.tensor([0,1]) if 1 in y else torch.tensor([1,0]), download=True)
    valset = DataClass(split='val', transform=transform, target_transform=lambda y:torch.tensor([0,1]) if 1 in y else torch.tensor([1,0]),  download=True)

    #  Split training set into `num_clients` partitions to simulate different local datasets
    partition_size = len(trainset) // num_clients
    if partition_size * num_clients == len(trainset):
        lengths = [partition_size] * num_clients 
    else: 
         lengths = [partition_size] * (num_clients-1)  + [partition_size+ (len(trainset) % num_clients)]
    train_datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))
    
    partition_size = len(valset) // num_clients
    if partition_size * num_clients == len(valset):
        lengths = [partition_size] * num_clients 
    else: 
         lengths = [partition_size] * (num_clients-1)  + [partition_size+ (len(valset) % num_clients)]
    # print(len(valset), lengths)
    val_datasets = random_split(valset, lengths, torch.Generator().manual_seed(42))
    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []

    for t_ds in train_datasets:
        trainloaders.append(DataLoader(t_ds, batch_size=batch_size, shuffle=True))


    for v_ds in val_datasets:
        valloaders.append(DataLoader(v_ds, batch_size=batch_size))
        
    testloader = DataLoader(testset, batch_size=batch_size)
    return trainloaders, valloaders, testloader



## Define Train and Test

In [39]:
from torchmetrics.classification import F1Score
f1 = F1Score(task="multiclass", num_classes=14)

def train(net, trainloader, epochs: int, verbose=False):
    """Train the network on the training set."""

    net.train()
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    # optimizer = torch.optim.Adam(net.parameters(), lr=0.1)

    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for i, batch in enumerate(trainloader):
            images, labels = batch
            optimizer.zero_grad()
            # images = images.expand(-1, 3, -1, -1)
            outputs = net(images)
            labels_ = labels.to(torch.float32)

            loss = criterion(outputs, labels_)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss

    return epoch_loss, 0.0

def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()

    accuracies = []
    with torch.no_grad():
        for batch in testloader:
            images, labels = batch
            # images = images.expand(-1, 3, -1, -1)
            # labels = labels.long()
            labels = labels.to(torch.float32)
            outputs = net(images)

            loss += criterion(outputs, labels).item()



            total += labels.size(0)
            # outputs = F.softmax(outputs, dim=1)

            predicted_classes = torch.argmax(outputs, dim=1)
            true_classes = torch.argmax(labels, dim=1)

            correct += torch.sum(predicted_classes == true_classes).item()



    loss /= len(testloader.dataset)
    # accuracy = metrics[1]
    accuracy = correct / total
    # accuracy = np.mean(accuracies)
    return loss, accuracy * 100          



In [40]:
# trainloaders, valloaders, testloader = load_datasets(3, batch_size=32)
# net = Net(in_channels=1, num_classes=14)
# train(net, trainloaders[0], 1)
# test(net, testloader)

## Model 

In [41]:

class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 14)


    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        # print(x.shape)
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = F.softmax(x, dim=1)
        return x


# class Net2(nn.Module):
#     def __init__(self) -> None:
#         super(Net2, self).__init__()
#         self.conv1 = nn.Conv2d(1, 6, 3)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(6, 16, 5)
#         self.fc1 = nn.Linear(6 * 13 * 13, 500)
#         self.fc2 = nn.Linear(500, 250)
#         self.fc3 = nn.Linear(250, 120)
#         self.fc4 = nn.Linear(120, 84)
#         # self.fc5 = nn.Linear(120, 84)
#         self.fc5 = nn.Linear(84, 2)


#     def forward(self, x: torch.Tensor) -> torch.Tensor:
#         x = self.pool(F.relu(self.conv1(x)))
#         # x = self.pool(F.relu(self.conv2(x)))

#         # print(x.shape)
#         x = x.view(-1, 6 * 13 * 13)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = F.relu(self.fc3(x))
#         x = F.relu(self.fc4(x))
#         x = self.fc5(x)
#         # x = F.softmax(x)
#         return x
# # Load model and data
# # net = Net().to(DEVICE)

In [42]:
class Net(nn.Module):
    def __init__(self, in_channels, num_classes)->None:
        super(Net, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels, 16, kernel_size=3),
            nn.BatchNorm2d(16),
            nn.ReLU())

        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 16, kernel_size=3),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer3 = nn.Sequential(
            nn.Conv2d(16, 64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU())
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU())

        self.layer5 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.fc = nn.Sequential(
            nn.Linear(64 * 4 * 4, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes))

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x



In [43]:

    
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict(
        {
            k: torch.Tensor(v) if v.shape != torch.Size([]) else torch.Tensor([0])
            for k, v in params_dict
        }
    )
    net.load_state_dict(state_dict, strict=True)


class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        # print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        # print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {"loss": float(loss), "accuracy": float(accuracy)}

    def evaluate(self, parameters, config):
        # print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"loss": float(loss), "accuracy": float(accuracy)}

def client_fn(cid) -> FlowerClient:
    # net = Net().to(DEVICE)
    net = Net(in_channels=1, num_classes=2).to(DEVICE)

    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

In [44]:

def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    # print(metrics)
    min_accuracy = min([m['accuracy'] for _, m in metrics])
    max_accuracy = max([m['accuracy'] for _, m in metrics])
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]
    # Aggregate and return custom metric (weighted average)
    # return {'accuracy': min(accuracies)}
    return {"average accuracy": sum(accuracies) / sum(examples), "min_accuracy": min_accuracy, "max_accuracy": max_accuracy}

In [45]:
# The `evaluate` function will be by Flower called after every round
def evaluate(
    server_round: int,
    parameters: fl.common.NDArrays,
    config: Dict[str, fl.common.Scalar],
) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    net = Net(in_channels=1, num_classes=2).to(DEVICE)
    
    # net = Net().to(DEVICE)
    set_parameters(net, parameters)  # Update model with the latest parameters
    loss, accuracy = test(net, testloader)
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
    return loss, {"accuracy": accuracy}

## Poisoning Attacks Implementation

In [46]:
def poison_labels(poisoned_dataset, poison_rate=0.25):
    # Calculate the number of samples to poison
    num_samples = len(poisoned_dataset)
    num_to_poison = int(num_samples * poison_rate)

    # Randomly select samples to poison
    indices_to_poison = np.random.choice(num_samples, num_to_poison, replace=False)
    
    for idx in indices_to_poison:
        poisoned_dataset[idx] = 1 - poisoned_dataset[idx]

def single_pixel_perturbations(poisoned_dataset, x, y, new_value, poison_rate=0.25):
    
    num_samples = len(poisoned_dataset)
    num_to_poison = int(num_samples * poison_rate)

    # Randomly select samples to poison
    indices_to_poison = np.random.choice(num_samples, num_to_poison, replace=False)
        
    for idx in indices_to_poison:
        poisoned_dataset[idx][0, y, x] = new_value

In [59]:
# Create an instance of the model and get the parameters
NUM_CLIENTS = 3
BATCH_SIZE = 32
NUM_ROUNDS = 3
random.seed(0)
torch.manual_seed(0)
trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS, batch_size=BATCH_SIZE)


CLIENT = 1  #0, 1, 2
Poison_type = poison_labels # poison_labels / single_pixel_perturbations
# Poison_type = single_pixel_perturbations
Poison_rate = 1

def modify_traindata(trainloaders, poison_type, poison_rate=1):
    if(poison_type == single_pixel_perturbations):

        for batch in (trainloaders):
            single_pixel_perturbations(batch[0], 10, 10, torch.max(batch[0]), poison_rate=poison_rate)
     
    else:
        for batch in (trainloaders):
            poison_labels(batch[1], poison_rate)

modify_traindata(trainloaders=trainloaders[0], poison_type=Poison_type, poison_rate=Poison_rate)
modify_traindata(trainloaders=trainloaders[1], poison_type=Poison_type, poison_rate=Poison_rate)
modify_traindata(trainloaders=trainloaders[2], poison_type=Poison_type, poison_rate=Poison_rate)

server_config = fl.server.ServerConfig(num_rounds=NUM_ROUNDS)
# Pass parameters to the Strategy for server-side parameter initialization
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    fraction_evaluate=1.0,
    min_fit_clients=NUM_CLIENTS,
    min_evaluate_clients=NUM_CLIENTS,
    min_available_clients=NUM_CLIENTS,
    # initial_parameters=fl.common.ndarrays_to_parameters(params),
    evaluate_metrics_aggregation_fn=weighted_average, 
    # fit_metrics_aggregation_fn=weighted_average,
    evaluate_fn=evaluate
    
)

# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=server_config,  
    strategy=strategy,
    client_resources=client_resources,
)

Using downloaded and verified file: C:\Users\Sama\.medmnist\chestmnist.npz
Using downloaded and verified file: C:\Users\Sama\.medmnist\chestmnist.npz
Using downloaded and verified file: C:\Users\Sama\.medmnist\chestmnist.npz


INFO flwr 2024-01-25 15:29:56,737 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2024-01-25 15:30:02,148	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-01-25 15:30:06,226 | app.py:213 | Flower VCE: Ray initialized with resources: {'object_store_memory': 1780618444.0, 'memory': 3561236891.0, 'node:127.0.0.1': 1.0, 'node:__internal_head__': 1.0, 'CPU': 16.0}
INFO flwr 2024-01-25 15:30:06,226 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2024-01-25 15:30:06,227 | app.py:227 | No `client_resources` specified. Using minimal resources for clients.
INFO flwr 2024-01-25 15:30:06,228 | app.py:242 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO flwr 2024-01-25 15:30:06,243 | app.py:288 | Flower VCE: Creating VirtualClientEngineActorPool with 16 actors
INFO flwr 2024-01-25 15:30:06,244 | server.py:89 | Ini

Server-side evaluation loss 0.02168928379360589 / accuracy 53.171666740961975


DEBUG flwr 2024-01-25 15:31:43,824 | server.py:236 | fit_round 1 received 3 results and 0 failures
WARNING flwr 2024-01-25 15:31:43,848 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
INFO flwr 2024-01-25 15:31:56,915 | server.py:125 | fit progress: (1, 0.021521533326809752, {'accuracy': 53.28756742299291}, 91.17829430000006)
DEBUG flwr 2024-01-25 15:31:56,916 | server.py:173 | evaluate_round 1: strategy sampled 3 clients (out of 3)


Server-side evaluation loss 0.021521533326809752 / accuracy 53.28756742299291


...

Server-side evaluation loss 0.019953382024262098 / accuracy 64.74390406989703


DEBUG flwr 2024-01-25 15:33:22,173 | server.py:187 | evaluate_round 2 received 3 results and 0 failures
DEBUG flwr 2024-01-25 15:33:22,174 | server.py:222 | fit_round 3: strategy sampled 3 clients (out of 3)
DEBUG flwr 2024-01-25 15:34:39,508 | server.py:236 | fit_round 3 received 3 results and 0 failures
INFO flwr 2024-01-25 15:34:52,612 | server.py:125 | fit progress: (3, 0.019616970776055558, {'accuracy': 65.68002496322383}, 266.8755622000001)
DEBUG flwr 2024-01-25 15:34:52,613 | server.py:173 | evaluate_round 3: strategy sampled 3 clients (out of 3)


Server-side evaluation loss 0.019616970776055558 / accuracy 65.68002496322383


DEBUG flwr 2024-01-25 15:34:57,071 | server.py:187 | evaluate_round 3 received 3 results and 0 failures
INFO flwr 2024-01-25 15:34:57,072 | server.py:153 | FL finished in 271.33536690000005
INFO flwr 2024-01-25 15:34:57,074 | app.py:226 | app_fit: losses_distributed [(1, 0.02149037970845535), (2, 0.019773867277535984), (3, 0.019501843008092566)]
INFO flwr 2024-01-25 15:34:57,074 | app.py:227 | app_fit: metrics_distributed_fit {}
INFO flwr 2024-01-25 15:34:57,075 | app.py:228 | app_fit: metrics_distributed {'average accuracy': [(1, 54.496929744697425), (2, 65.7634666249726), (3, 66.25371278277757)], 'min_accuracy': [(1, 53.96952686447474), (2, 65.47205135062852), (3, 65.63252206472319)], 'max_accuracy': [(1, 54.88098422037978), (2, 66.22091468307035), (3, 67.07675849157528)]}
INFO flwr 2024-01-25 15:34:57,075 | app.py:229 | app_fit: losses_centralized [(0, 0.02168928379360589), (1, 0.021521533326809752), (2, 0.019953382024262098), (3, 0.019616970776055558)]
INFO flwr 2024-01-25 15:34:57

History (loss, distributed):
	round 1: 0.02149037970845535
	round 2: 0.019773867277535984
	round 3: 0.019501843008092566
History (loss, centralized):
	round 0: 0.02168928379360589
	round 1: 0.021521533326809752
	round 2: 0.019953382024262098
	round 3: 0.019616970776055558
History (metrics, distributed, evaluate):
{'average accuracy': [(1, 54.496929744697425), (2, 65.7634666249726), (3, 66.25371278277757)], 'min_accuracy': [(1, 53.96952686447474), (2, 65.47205135062852), (3, 65.63252206472319)], 'max_accuracy': [(1, 54.88098422037978), (2, 66.22091468307035), (3, 67.07675849157528)]}History (metrics, centralized):
{'accuracy': [(0, 53.171666740961975), (1, 53.28756742299291), (2, 64.74390406989703), (3, 65.68002496322383)]}

In [48]:

# trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS, batch_size=BATCH_SIZE)
# old_train = trainloaders[1]
# modify_traindata(trainloaders=trainloaders[CLIENT], poison_type=single_pixel_perturbations, poison_rate=Poison_rate)
# new_train = trainloaders[1]
# for batch in old_train: 
#     print(batch[0][10])
#     break
# for batch in new_train: 
#     print(batch[0][10])
#     break

In [49]:
# 32 batch 
# lr = 1e-3 
# History (loss, distributed):
# 	round 1: 0.021634859852918794
# 	round 2: 0.019494445754617267
# 	round 3: 0.019551336893341268
# History (loss, centralized):
# 	round 0: 0.021838079879740166
# 	round 1: 0.021617193900751844
# 	round 2: 0.01964001085544267
# 	round 3: 0.01949963099403961
# History (metrics, distributed, evaluate):
# {'accuracy': [(1, 50.41448023590991), (2, 66.07548348123696), (3, 66.43182769728148)]}History (metrics, centralized):
# {'accuracy': [(0, 46.828333259038025), (1, 51.04979271608791), (2, 65.35461150982927), (3, 66.7186733829626)]}


# History (loss, distributed):
# 	round 1: 0.021248942525614416
# 	round 2: 0.019625080685606092
# 	round 3: 0.019320327407663318
# History (loss, centralized):
# 	round 0: 0.02166293802515575
# 	round 1: 0.02130382649956877
# 	round 2: 0.01972372806997405
# 	round 3: 0.019418413398981423
# History (metrics, distributed, evaluate):
# {'accuracy': [(1, 62.56357268082011), (2, 66.00401064781263), (3, 66.9222168388823)]}History (metrics, centralized):
# {'accuracy': [(0, 53.171666740961975), (1, 61.4897695359515), (2, 65.49280078455845), (3, 66.57602638969375)]}

In [50]:
# 64 batch 
# lr = 1e-3 

# History (loss, distributed):
# 	round 1: 0.010716385236471615
# 	round 2: 0.00979215710741931
# 	round 3: 0.009725512929264912
# History (loss, centralized):
# 	round 0: 0.010840601371187312
# 	round 1: 0.010640637767685297
# 	round 2: 0.009730266535249719
# 	round 3: 0.009642307873539136
# History (metrics, distributed, evaluate):
# {'accuracy': [(1, 65.28216410359872), (2, 66.73495334450656), (3, 66.7172567167055)]}History (metrics, centralized):
# {'accuracy': [(0, 53.171666740961975), (1, 64.39620202380422), (2, 66.59831498239201), (3, 66.89698212454866)]}

In [51]:
# 8 batch 
# lr = 1e-3 
# History (loss, distributed):
# 	round 1: 0.08661153506258784
# 	round 2: 0.07887566295962155
# 	round 3: 0.07774493999581282
# History (loss, centralized):
# 	round 0: 0.0864690203638629
# 	round 1: 0.08652663819157412
# 	round 2: 0.0792080924776444
# 	round 3: 0.07774718131632395
# History (metrics, distributed, evaluate):
# {'accuracy': [(1, 50.27183483503256), (2, 65.79912201753378), (3, 66.44974356690285)]}History (metrics, centralized):
# {'accuracy': [(0, 53.171666740961975), (1, 50.30981143850577), (2, 65.2877457317345), (3, 66.4244639593456)]}

In [52]:
#32 
# lr = 0.1 

# History (loss, distributed):
# 	round 1: 0.02237011717661033
# 	round 2: 0.021585349851597994
# 	round 3: 0.021599878324354702
# History (loss, centralized):
# 	round 0: 0.021675996097438352
# 	round 1: 0.02258573357783557
# 	round 2: 0.02163072222347155
# 	round 3: 0.021635136129023617
# History (metrics, distributed, evaluate):
# {'accuracy': [(1, 54.18494148519223), (2, 54.18494148519223), (3, 54.18494148519223)]}History (metrics, centralized):
# {'accuracy': [(0, 53.171666740961975), (1, 53.171666740961975), (2, 53.171666740961975), (3, 53.171666740961975)]}

In [53]:
# poison labels 
# rate = 1 
# History (loss, distributed):
# 	round 1: 0.021746917593043886
# 	round 2: 0.020342148526587642
# 	round 3: 0.019321635498937464
# History (loss, centralized):
# 	round 0: 0.02170411833565569
# 	round 1: 0.021733941010719307
# 	round 2: 0.020360666946800884
# 	round 3: 0.019343852770980552
# History (metrics, distributed, evaluate):
# {'accuracy': [(1, 45.81505851480777), (2, 65.97729412555603), (3, 66.5209876262538)]}History (metrics, centralized):
# {'accuracy': [(0, 46.828333259038025), (1, 46.828333259038025), (2, 65.52400481433602), (3, 66.5626532340748)]}

In [54]:
# SSP 
# rate = 1 
# History (loss, distributed):
# 	round 1: 0.021658491245307273
# 	round 2: 0.01963736343005101
# 	round 3: 0.019270213384083426
# History (loss, centralized):
# 	round 0: 0.02176191804614467
# 	round 1: 0.021650527516035453
# 	round 2: 0.01976930641970696
# 	round 3: 0.019276142710216947
# History (metrics, distributed, evaluate):
# {'accuracy': [(1, 50.53925266230466), (2, 66.13783633154864), (3, 67.19848298005492)]}History (metrics, centralized):
# {'accuracy': [(0, 46.828333259038025), (1, 51.01858868631035), (2, 65.43485044354298), (3, 66.83011634645388)]}

In [55]:
# 5 rounds trials 
# History (loss, distributed):
# 	round 1: 0.02150226518430445
# 	round 2: 0.01944696508263232
# 	round 3: 0.01931844058592802
# 	round 4: 0.019155672249873926
# 	round 5: 0.019090482891622185
# History (loss, centralized):
# 	round 0: 0.02178466496197426
# 	round 1: 0.02150007961363149
# 	round 2: 0.019510929829904367
# 	round 3: 0.01935488031444338
# 	round 4: 0.019111589488059365
# 	round 5: 0.019015373670002132
# History (metrics, distributed, evaluate):
# {'accuracy': [(1, 58.22254889253449), (2, 67.06476691722908), (3, 66.97566418177512), (4, 67.40351936025618), (5, 67.86701563291885)]}History (metrics, centralized):
# {'accuracy': [(0, 46.828333259038025), (1, 57.80323630365979), (2, 66.46012570766283), (3, 66.65626532340748), (4, 67.56118218695671), (5, 67.91779967012883)]}

In [56]:
# Poison labels 5 rounds 
# History (loss, distributed):
# 	round 1: 0.02132637792728052
# 	round 2: 0.019729355985695637
# 	round 3: 0.019349942204867
# 	round 4: 0.019245311957510104
# 	round 5: 0.019136189577066257
# History (loss, centralized):
# 	round 0: 0.021630475113821566
# 	round 1: 0.021393116159859532
# 	round 2: 0.019856321979333187
# 	round 3: 0.01943236189153613
# 	round 4: 0.019296907409590513
# 	round 5: 0.01909124156685098
# History (metrics, distributed, evaluate):
# {'accuracy': [(1, 55.06741602567626), (2, 65.63865606956561), (3, 66.62800623132914), (4, 67.08265895621781), (5, 66.99351332562507)]}History (metrics, centralized):
# {'accuracy': [(0, 53.171666740961975), (1, 53.8849017073062), (2, 64.89100878170552), (3, 66.2327820621406), (4, 66.8746935318504), (5, 67.72166005438417)]}

In [57]:
# SSP
# rate = 1 
# History (loss, distributed):
# 	round 1: 0.021477125957758282
# 	round 2: 0.01956604122161247
# 	round 3: 0.01929507566124522
# 	round 4: 0.01911810000906677
# 	round 5: 0.01906080256308064
# History (loss, centralized):
# 	round 0: 0.02168635337493999
# 	round 1: 0.021490031688832855
# 	round 2: 0.0196927915761215
# 	round 3: 0.0193582001228432
# 	round 4: 0.019169284116566616
# 	round 5: 0.019066446862282194
# History (metrics, distributed, evaluate):
# {'accuracy': [(1, 56.422216088217375), (2, 66.2091328182914), (3, 66.81517440317431), (4, 67.79567148491078), (5, 67.68870054110073)]}History (metrics, centralized):
# {'accuracy': [(0, 53.171666740961975), (1, 55.984487139482006), (2, 65.57749743681184), (3, 66.43337939642491), (4, 67.29817679311728), (5, 67.42299291222751)]}